In [1]:
# pip install Phrank

# from phrank import Phrank
# from phrank import utils as phrank_utils


# DAG=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\hpodag.txt"
# DISEASE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\disease_to_pheno.build127.txt"
# DISEASE_TO_GENE=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_disease.build127.txt"
# GENE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_pheno.amelie.txt"
# p_hpo = Phrank(DAG, diseaseannotationsfile=DISEASE_TO_PHENO, diseasegenefile=DISEASE_TO_GENE)


# # defining the phenotype sets
# phenotypeset1 = ['HP:0000077','HP:0030765','HP:0012115','HP:0002088','HP:0002099','HP:0001945','HP:0000719']
# phenotypeset2 = ['HP:0000975','HP:0002018','HP:0000421','HP:0012393','HP:0004406','HP:0002321']

# # computing the similarity between two sets of phenotypes
# matchscore = p_hpo.compute_phenotype_match(phenotypeset1, phenotypeset2)

# print ("the phenotype similarity score is %.2f"%matchscore)


# # defining patient genes and phenotypes
# patient_genes = set(['ENSG00000000419','ENSG00000000971','ENSG00000000971','ENSG00000001626','ENSG00000001626','ENSG00000001631','ENSG00000002822','ENSG00000003137'])
# patient_phenotypes = phenotypeset1

# # sorting the disease by best match
# disease_ranking = p_hpo.rank_diseases([],patient_phenotypes)
# print ("\nDisease ranking")
# for disease_info in disease_ranking:
#     print ("disease id: %s\tsimilarity score: %.2f"%(disease_info[1],disease_info[0]))

# # sorting the genes by best match
# gene_ranking = p_hpo.rank_genes(patient_genes, patient_phenotypes)
# print ("\nGene ranking")
# for gene_info in gene_ranking:
#     print ("ensembl gene id: %s\tsimilarity score: %.2f"%(gene_info[1],gene_info[0]))



# def rank_diseases(self, patient_phenotypes, baseline=False):
#         """Compute the Phrank score for each disease matching the patient phenotypes"""
#         disease_scores = []
#         for disease in self._disease_pheno_map:
#             if self._disease_gene_map[disease]:
#                 disease_phenos = self._disease_pheno_map.get(disease, set([]))
#                 score = self.compute_phenotype_match(patient_phenotypes, disease_phenos)
#                 score = self.compute_phenotype_match(patient_phenotypes, disease_phenos) if not baseline else self.compute_baseline_match(patient_phenotypes, disease_phenos)
#                 disease_scores.append((score, disease))
#         disease_scores.sort(reverse=True)
#         return disease_scores

# from collections import defaultdict
# import math
# from .utils import load_maps, load_term_hpo, closure, load_disease_gene, compute_gene_disease_pheno_map

# class Phrank:
#     @staticmethod
#     def compute_information_content(annotations_map, child_to_parent_map):
#         information_content, marginal_information_content = {}, {}
#         annotatedGeneCt = 0
#         associated_phenos = defaultdict(set)
#         for gene, phenos in annotations_map.items():
#             annotatedGeneCt += 1
#             all_ancestors = closure(phenos, child_to_parent_map)

#             # for each ancestor increment the count since this pheno is now associated with the specified gene
#             for pheno in all_ancestors:
#                 associated_phenos[pheno].add(gene)

#         phenos = associated_phenos.keys()
#         for pheno in phenos:
#             information_content[pheno] = -math.log(1.0*len(associated_phenos[pheno])/annotatedGeneCt, 2) if len(associated_phenos[pheno]) else 0

#         for pheno in phenos:
#             parent_phenos = child_to_parent_map[pheno]
#             parent_entropy = 0
#             if len(parent_phenos) == 1:
#                 parent_entropy = information_content[parent_phenos[0]]
#             elif len(parent_phenos) > 1:
#                 list_of_phenosets = [associated_phenos[parent] for parent in parent_phenos]
#                 parent_set = set([])
#                 for phenoset in list_of_phenosets:
#                     parent_set = parent_set | phenoset if parent_set else phenoset
#                 parent_entropy = -math.log(1.0*len(parent_set)/annotatedGeneCt, 2) if len(parent_set) else 0
#             marginal_information_content[pheno] = information_content[pheno] - parent_entropy
#         return information_content, marginal_information_content

#     def __init__(self, dagfile, diseaseannotationsfile=None, diseasegenefile=None, geneannotationsfile=None):
#         """Initialize Phrank object with the disease annotations file or gene annotations file"""
#         self._child_to_parent, self._parent_to_children = load_maps(dagfile)
#         if diseaseannotationsfile and diseasegenefile:
#             self._disease_pheno_map = load_term_hpo(diseaseannotationsfile)
#             self._disease_gene_map = load_disease_gene(diseasegenefile)
#             self._gene_pheno_map = compute_gene_disease_pheno_map(self._disease_gene_map, self._disease_pheno_map)
#             self._IC, self._marginal_IC = Phrank.compute_information_content(self._gene_pheno_map, self._child_to_parent)
#             self._gene_and_disease  = True
#         elif geneannotationsfile:
#             self._gene_pheno_map = load_term_hpo(geneannotationsfile)
#             self._IC, self._marginal_IC = Phrank.compute_information_content(self._gene_pheno_map, self._child_to_parent)
#             self._gene_and_disease = False
    
#     def get_causal_rank(self, scores, causal_item):
#         rank = 1
#         for score in scores:
#             if score[1] in causal_item:
#                 return score[1], score[0], rank
#             else:
#                 rank = rank + 1
#         return causal_item, 0, len(scores) + 1

#     # def rank_diseases(self, patient_genes, patient_phenotypes, baseline=False):
#     #     """Compute the Phrank score for each disease matching the patient phenotypes"""
#     #     disease_scores = []
#     #     for disease in self._disease_pheno_map:
#     #         if self._disease_gene_map[disease] & patient_genes:
#     #             disease_phenos = self._disease_pheno_map.get(disease, set([]))
#     #             score = self.compute_phenotype_match(patient_phenotypes, disease_phenos)
#     #             score = self.compute_phenotype_match(patient_phenotypes, disease_phenos) if not baseline else self.compute_baseline_match(patient_phenotypes, disease_phenos)
#     #             disease_scores.append((score, disease))
#     #     disease_scores.sort(reverse=True)
#     #     return disease_scores

#     def rank_diseases(self, patient_phenotypes, baseline=False):
#         """Compute the Phrank score for each disease matching the patient phenotypes"""
#         disease_scores = []
#         for disease in self._disease_pheno_map:
#             if self._disease_gene_map[disease]:
#                 disease_phenos = self._disease_pheno_map.get(disease, set([]))
#                 score = self.compute_phenotype_match(patient_phenotypes, disease_phenos)
#                 score = self.compute_phenotype_match(patient_phenotypes, disease_phenos) if not baseline else self.compute_baseline_match(patient_phenotypes, disease_phenos)
#                 disease_scores.append((score, disease))
#         disease_scores.sort(reverse=True)
#         return disease_scores

#     def rank_genes(self, patient_genes, patient_phenotypes, normalized=False, baseline=False):
#         gene_scores = []
#         if self._gene_and_disease:
#             gene_scores = self.rank_genes_using_disease(patient_genes, patient_phenotypes, normalized=normalized, baseline=baseline)
#         else:
#             gene_scores = self.rank_genes_directly(patient_genes, patient_phenotypes, normalized=normalized, baseline=baseline)
#         return gene_scores

#     def rank_genes_directly(self, patient_genes, patient_phenotypes, normalized=False, baseline=False):
#         gene_scores = []
#         for gene in patient_genes:
#             gene_phenos = self._gene_pheno_map[gene]
#             score = self.compute_phenotype_match(patient_phenotypes, gene_phenos) if not baseline else self.compute_baseline_match(patient_phenotypes, gene_phenos)
#             if normalized:
#                 max_gene_score = self.compute_maximal_match(gene_phenos)
#                 score = 1.0*score/max_gene_score
#             gene_scores.append((score, gene))
#         gene_scores.sort(reverse=True)
#         return gene_scores

#     def rank_genes_using_disease(self, patient_genes, patient_phenotypes, normalized=False, baseline=False):
#         """Compute the Phrank score for each gene matching the patient phenotypes"""
#         genedisease_scores = defaultdict(list)
#         for disease in self._disease_pheno_map:
#             if self._disease_gene_map[disease] & patient_genes:
#                 disease_phenos = self._disease_pheno_map.get(disease, set([]))
#                 score = self.compute_phenotype_match(patient_phenotypes, disease_phenos) if not baseline else self.compute_baseline_match(patient_phenotypes, disease_phenos)
#                 if normalized:
#                     max_disease_score = self.compute_maximal_disease_match(disease)
#                     score = 1.0*score/max_disease_score
#                 for gene in self._disease_gene_map[disease] & patient_genes:
#                     genedisease_scores[gene].append(score)

#         gene_scores = []
#         for gene in genedisease_scores:
#             score = max(genedisease_scores[gene])
#             gene_scores.append((score, gene))
#         gene_scores.sort(reverse=True)
#         return gene_scores

#     def compute_maximal_match(self, phenotypes):
#         """
#         input: a list of phenotypes [HPO:XXX, HPOYYY] 
#         output: score (Float) - the maximal score this set of phenotypes can receive

#         The maximum score achievable for a set of phenotypes is if every phenotype
#         is found in the comparing set. This number can be used to normalize the phenotype 
#         score so that query with a large # of phenotypes do not skew the results
#         """
#         return self.compute_phenotype_match(phenotypes, phenotypes)

#     def compute_maximal_disease_match(self, disease):
#         disease_phenos = self._disease_pheno_map.get(disease, set([]))
#         return self.compute_phenotype_match(disease_phenos, disease_phenos)

#     def compute_phenotype_match(self, patient_phenotypes, query_phenotypes):
#         """
#         input: patient_phenotypes, query phenotypes - two lists of phenotypes
#         output: score - Phrank score measuring the similarity between the two sets
#         """
#         #change_to_primary, patient_genes, disease_gene_map, disease_pheno_map, child_to_parent, disease_marginal_content)
#         all_patient_phenotypes = closure(patient_phenotypes, self._child_to_parent)
#         all_query_phenotypes = closure(query_phenotypes, self._child_to_parent)
#         similarity_score = 0
#         for phenotype in all_patient_phenotypes & all_query_phenotypes:
#             similarity_score += self._marginal_IC.get(phenotype, 0)
#         return similarity_score
   
#     def compute_baseline_match(self, patient_phenotypes, query_phenotypes):
#         all_patient_phenotypes = closure(patient_phenotypes, self._child_to_parent)
#         all_query_phenotypes = closure(query_phenotypes, self._child_to_parent)
#         return len(all_patient_phenotypes & all_query_phenotypes)


# from collections import defaultdict
# def load_maps(human_phenotype_map_file):
#     hpo_file = open(human_phenotype_map_file)
#     child_to_parent = defaultdict(list)
#     parent_to_children = defaultdict(list)
#     for hpo_line in hpo_file:
#         hpo_tokens = hpo_line.strip().split("\t")
#         child = hpo_tokens[0]
#         parent = hpo_tokens[1]
#         child_to_parent[child].append(parent)
#         parent_to_children[parent].append(child)
#     return child_to_parent, parent_to_children

# def load_term_hpo(term_to_hpo_file):
#     term_hpo_file = open(term_to_hpo_file)
#     term_pheno_map = defaultdict(list)
#     for term_line in term_hpo_file:
#         term_hpo_tokens = term_line.strip().split("\t")
#         hpo = term_hpo_tokens[0]
#         term = term_hpo_tokens[1]
#         term_pheno_map[term].append(hpo)
#     term_hpo_file.close()
#     return term_pheno_map

# def closure(phenos, child_to_parent):
#     all_ancestors = set([])
#     for pheno in phenos:
#         all_ancestors = all_ancestors | set(get_all_ancestors(pheno, child_to_parent)) | set([pheno])
#     return all_ancestors

# def get_all_ancestors(hpo_term, child_to_parent_map):
#     ancestors = []
#     term = hpo_term
#     parents = child_to_parent_map.get(term, [])[:]
#     while parents:
#         parent = parents.pop()
#         ancestors.append(parent)
#         parents = parents + child_to_parent_map.get(parent, [])
#     return ancestors

# def compute_gene_disease_pheno_map(disease_gene_map, disease_pheno_map):
#     gene_pheno_map = defaultdict(set)    
#     for disease, genes in disease_gene_map.items():
#         phenos = disease_pheno_map.get(disease)
#         for gene in genes:
#             for pheno in phenos:
#                 gene_pheno_map[gene].add(pheno)
#     return gene_pheno_map

# def load_disease_gene(disease_to_gene_filename):
#     disease_to_gene = defaultdict(set)
#     f = open(disease_to_gene_filename)
#     for line in f:
#         tokens = line.strip().split("\t")
#         gene = tokens[0]
#         disease = tokens[1]
#         disease_to_gene[disease].add(gene)
#     return disease_to_gene

# def load_gene_symbol_map(GENE_TO_SYMBOL):
#     gene_to_symbol_map = {}
#     f = open(GENE_TO_SYMBOL)
#     for line in f:
#         gene_data = line.strip().split("\t")
#         gene_to_symbol_map[gene_data[0]] = gene_data[1]
#     return gene_to_symbol_map

# pip uninstall Phrank

In [ ]:
# import pandas as pd

# file_path = r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\hpo_disorders.csv"

# def filter_hpo_data(file_path, symptoms=None, frequency=None):
   
#     # Load the dataset
#     df = pd.read_csv(file_path)
    
#     # Apply filters
#     if symptoms:
#         df = df[df["HPO Term"].str.contains('|'.join(symptoms), case=False, na=False)]
#     if frequency:
#         df = df[df["Frequency"].isin(frequency)]    
#     return df

# # Example usage
# # file_path = "/mnt/data/hpo_disorders.csv"
# symptoms = ["Headache","Seizure"]
# frequency = ["Very frequent (99-80%)","Very frequent (99-80%)"]
# filtered_df = filter_hpo_data(file_path, symptoms, frequency)
# len(filtered_df)

# set(list(filtered_df["HPO ID"]))

# from phrank_modified import Phrank
# from phrank_modified import utils

# DAG=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\hpodag.txt"
# DISEASE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\disease_to_pheno.build127.txt"
# DISEASE_TO_GENE=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_disease.build127.txt"
# GENE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_pheno.amelie.txt"
# p_hpo = Phrank(DAG, diseaseannotationsfile=DISEASE_TO_PHENO, diseasegenefile=DISEASE_TO_GENE)


# # defining the phenotype sets
# # phenotypeset1 = ['HP:0000077','HP:0030765','HP:0012115','HP:0002088','HP:0002099','HP:0001945','HP:0000719']
# # phenotypeset2 = ['HP:0000975','HP:0002018','HP:0000421','HP:0012393','HP:0004406','HP:0002321']
# ourpheno=set(list(filtered_df["HPO ID"]))

# # computing the similarity between two sets of phenotypes
# matchscore = p_hpo.compute_phenotype_match(phenotypeset1, phenotypeset2)

# print ("the phenotype similarity score is %.2f"%matchscore)


# # defining patient genes and phenotypes
# # patient_genes = set(['ENSG00000000419','ENSG00000000971','ENSG00000000971','ENSG00000001626','ENSG00000001626','ENSG00000001631','ENSG00000002822','ENSG00000003137'])
# patient_phenotypes = ourpheno

# # sorting the disease by best match
# disease_ranking = p_hpo.rank_diseases(patient_phenotypes)
# print ("\nDisease ranking")
# for disease_info in disease_ranking:
#     print ("disease id: %s\tsimilarity score: %.2f"%(disease_info[1],disease_info[0]))

# # # sorting the genes by best match
# # gene_ranking = p_hpo.rank_genes(patient_genes, patient_phenotypes)
# # print ("\nGene ranking")
# # for gene_info in gene_ranking:
# #     print ("ensembl gene id: %s\tsimilarity score: %.2f"%(gene_info[1],gene_info[0]))

In [1]:
pip install Phrank


  Using cached phrank-1.6-py3-none-any.whl


In [9]:
import pandas as pd

file_path = r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\hpo_disorders.csv"

def filter_hpo_data(file_path, symptoms=None, frequency=None):
   
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Apply filters
    if symptoms:
        df = df[df["HPO Term"].str.contains('|'.join(symptoms), case=False, na=False)]
    if frequency:
        df = df[df["Frequency"].isin(frequency)]    
    return df

# Example usage
# file_path = "/mnt/data/hpo_disorders.csv"
symptoms = ["Headache","Seizure"]
frequency = ["Very frequent (99-80%)","Very frequent (99-80%)"]
filtered_df = filter_hpo_data(file_path, symptoms, frequency)
len(filtered_df)

set(list(filtered_df["HPO ID"]))

from phrank import Phrank
from phrank import utils as phrank_utils

DAG=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\hpodag.txt"
DISEASE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\disease_to_pheno.build127.txt"
DISEASE_TO_GENE=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_disease.build127.txt"
GENE_TO_PHENO=r"C:\Users\1842s\Documents\Studies\HarvardRareDiseaseHackathon\Phrank\demo\data\gene_to_pheno.amelie.txt"
p_hpo = Phrank(DAG, diseaseannotationsfile=DISEASE_TO_PHENO, diseasegenefile=DISEASE_TO_GENE)


# defining the phenotype sets
# phenotypeset1 = ['HP:0000077','HP:0030765','HP:0012115','HP:0002088','HP:0002099','HP:0001945','HP:0000719']
# phenotypeset2 = ['HP:0000975','HP:0002018','HP:0000421','HP:0012393','HP:0004406','HP:0002321']
ourpheno=set(list(filtered_df["HPO ID"]))

# computing the similarity between two sets of phenotypes
# matchscore = p_hpo.compute_phenotype_match(phenotypeset1, phenotypeset2)

# print ("the phenotype similarity score is %.2f"%matchscore)


# defining patient genes and phenotypes
patient_genes = set(['ENSG00000001626','ENSG00000002822','ENSG00000003137']) # 'ENSG00000001631'
patient_phenotypes = ourpheno

# sorting the disease by best match
disease_ranking = p_hpo.rank_diseases(patient_genes,patient_phenotypes)
print ("\nDisease ranking")
for disease_info in disease_ranking:
    print ("disease id: %s\tsimilarity score: %.2f"%(disease_info[1],disease_info[0]))

# sorting the genes by best match
gene_ranking = p_hpo.rank_genes(patient_genes, patient_phenotypes)
print ("\nGene ranking")
for gene_info in gene_ranking:
    print ("ensembl gene id: %s\tsimilarity score: %.2f"%(gene_info[1],gene_info[0]))


Disease ranking
disease id: OMIM:219700	similarity score: 5.90
disease id: OMIM:614416	similarity score: 0.71
disease id: OMIM:277180	similarity score: 0.00
disease id: OMIM:176807	similarity score: 0.00

Gene ranking
ensembl gene id: ENSG00000001626	similarity score: 5.90
ensembl gene id: ENSG00000003137	similarity score: 0.71
ensembl gene id: ENSG00000002822	similarity score: 0.00
